# Cryptography

## Public key cryptography - Part 4

# RSA Signatures, RSA Encryption, Factoring

Formuation of propositions, lemmas, definitions comes from:
**Introduction to modern cryptography** by *Jonathan Katz* and *Yehuda Lindell*

To see the notebook as a slide-show, use [RISE](https://rise.readthedocs.io/en/stable/)

In [1]:
import math
import random
from sympy import randprime, isprime, Mod

## Agenda

### RSA

* Encryption (OAEP)
* Signatures

### Factoring algorithms

* trial division
* p-1 method
* fermat's factoring method
* QS - quadratic sieve 
* ECM - elliptic curve method
* NFS - number field sieve

**Proposition 7.2** Let $a, b$ be positive integers. Then there exists integers $X, Y$ such that $Xa + Yb = \gcd(a, b)$. 

Furthermore, $\gcd(a, b)$ is the smallest positive integer that can be expressed in this way.

In [2]:
def egcd(a, b):
    if a == 0:
        return (b, 0, 1)
    else:
        g, y, x = egcd(b % a, a)
        return (g, x - (b // a) * y, y)

def modinv(a, m):
    g, x, y = egcd(a, m)
    if g != 1:
        raise Exception('modular inverse does not exist')
    else:
        return x % m

In [13]:
a = 10
b = 21

gcd, x, y = egcd(a, b)

print(gcd)

a_min_1 = modinv(a, b)
print(a_min_1)
a_min_1 * 10 % b

1
19


1

In [ ]:
print(a, "*", x, "+", b, "*", y, "=", math.gcd(a,b))
a * x + b * y

If for $b$ there exists an integer $b^{-1}$ such that $b b^{-1} = 1 \bmod N$ then we call:

* $b$ invertible

* $b^{-1}$ modular inverse of $b$ modulo $N$

**Proposition 7.7** Let $a, N$ be integers, with $N > 1$. Then $a$ is invertible modulo $N$ if and only if $\gcd(a, N) = 1$.

**Definition 7.9** A group is a set $\mathbf{G}$ along with a binary operation $\oplus$ for which the following conditions hold:

* (Closure:) For all $g, h \in \mathbf{G}$, $g \oplus h \in \mathbf{G}$

* (Existance of an Identity:) There exists an **identity** $e \in \mathbf{G}$ such that for all $g \in \mathcal{G}$, $e \oplus g = g = g \oplus e$.

* (Existence of Inverses:) For all $g \in \mathbf{G}$ there exists an element $h \in \mathbf G$ such that $g \oplus h = e$

* (Associativity:) For all $g_1, g_2, g_3 \in \mathbf{G}$, $(g_1 \oplus g_2) \oplus g_3 = g_1 \oplus (g_2 \oplus g_3)$.

If $\mathbf{G}$ has finite number of elements $\rightarrow \mathbf{G}$ is a *finite group*

$|\mathbf{G}|$ - order of the group - number of elements in $\mathbf{G}$

$\mathbf{G}$ is **abelian** if
for all $g, h \in \mathbf{G}, g \oplus h = h \oplus g$

$Z_n^* = (\{a: \gcd(a, n) = 1\}, \cdot_{ \bmod n})$. 

Let $n = p \cdot q$, $\qquad p, q$ are primes -- an abelian group of order $|Z_{pq}^*| = (p - 1)(q-1)$

**The factoring experiment** $\textsf{Factor}_{\mathcal{A}, GenModulus}(n)$
1. $(N, p, q) := GenModulus(1^n)$

2. $(p', q') = \mathcal{A}(N)$

3. Output: 1 if $p' \cdot q' = N$ and $p', q' > 1$

**Definition 7.45** We say that factoring is hard relative to **GenModulus** if for all probabilistic polynomial-time algorithms $\mathsf{A}$ there exists a negligible function **negl** such that 
$$P[\textsf{Factor}_{\mathcal{A}, GenModulus}(n) = 1] \leq negl(n)$$

In [14]:
from sympy import primefactors

**The factoring experiment** $\textsf{Factor}_{\mathcal{A}, GenModulus}(n)$
1. $(N, p, q) := GenModulus(1^n)$

2. $(p', q') = \mathcal{A}(N)$

3. Output: 1 if $p' \cdot q' = N$ and $p', q' > 1$

**Definition 7.45** We say that factoring is hard relative to **GenModulus** if for all probabilistic polynomial-time algorithms $\mathsf{A}$ there exists a negligible function **negl** such that 
$$P[\textsf{Factor}_{\mathcal{A}, GenModulus}(n) = 1] \leq negl(n)$$

In [16]:
def GenModulus(w):
    n = len(w)
    p = randprime(2 ** n, 2 ** (n+1))
    q = randprime(2 ** n, 2 ** (n+1))
    N = p * q
    return N, p, q

In [17]:
def Factor(n):
    N, p, q = GenModulus("1" * n)
    pp, qq = A(N)
    if pp > 1 and qq > 1 and pp * qq == N:
        return 1
    else:
        return 0

In [18]:
def A(N):
    factors = primefactors(N)
    return factors[0], factors[1]

In [22]:
%%time
sec_param = 52
num_of_succ = 0
tries = 100

for i in range(tries):
    num_of_succ = num_of_succ + Factor(sec_param // 2)

num_of_succ/tries

CPU times: user 12.4 s, sys: 0 ns, total: 12.4 s
Wall time: 12.4 s


1.0

In [42]:
def GenRSA(w):
    n = len(w)
    N, p, q = GenModulus(w)
    m = (p-1) * (q-1)
    e = 2 ** 16 + 1
    d = modinv(e, m)
    return N, e, d, p, q
# N, e - public key
# N, d - private key

def enc(x, N, e):
    return x ** e % N

def dec(c, N, d):
    return c ** d % N

N, e, d, p, q = GenRSA("111111111")
#print("private key: ", d)
#print("public key: ", N, e)

xx  = 120 * p % N #random.randint(2, N)

y = enc(xx, N, e)
x = dec(y, N, d)
print(N, e, y)

488489 65537 192722


**The RSA experiment** $\textsf{RSA-inv}_{\mathcal{A}, GenRSA}(n)$
1. $(N, e, d) := GenRSA(1^n)$

2. choose $y \leftarrow Z_N^*$

2. $x = \mathcal{A}(N, e, y)$

3. Output: 1 if $x^e = y \bmod N$

**Definition 7.46** We say that RSA problem is hard relative to **GenRSA** if for all probabilistic polynomial-time algorithms $\mathsf{A}$ there exists a negligible function **negl** such that 
$$P[\textsf{RSA-inv}_{\mathcal{A}, GenRSA}(n) = 1] \leq negl(n)$$

In [43]:
def randomZnElement(N):
    g = N
    while math.gcd(g, N) != 1:
        g = random.randint(2, N)
    return g

### CPA security $Pub_{\mathcal{A}, \Pi}^{cpa}(n):$

1. $(pk, sk) \leftarrow Gen(1^n)$
2. * $\mathcal{A}$ is given $pk$ and oracle access to $Enc_{pk}(\cdot)$
    * outputs $m_0, m_1 \leftarrow \mathcal{A}(pk)$, $|m_0|=|m_1|$
3. 
    * $b \leftarrow \{0, 1\}$
    * $c \leftarrow Enc_{pk}(m_b)$
4. * $\mathcal{A}$ continues to have access to $Enc_{pk}(\cdot)$ and 
    * outputs $b' \leftarrow \mathcal{A}(c)$
5. output is 1 if $b' = b$, 0 otherwise

**Definition 10.4** A public-key encryption scheme $\Pi = (Gen, Enc, Dec)$ has **indistinguishable encryptions under a chosen-plaintext attack** (CPA-secure) if for all probabilistic polynomial-time adversaries $\mathcal{A}$ there exists a negligible function **negl** such that:
$$P\left[Pub_{\mathcal{A}, \Pi}^{cpa}(n) = 1\right] \leq \frac{1}{2} + \textsf{negl}(n)$$

Is "textbook RSA" CPA-secure?

it is deterministic so...

In [54]:
print("pub",N, e)
m0 = randomZnElement(N)
m1 = randomZnElement(N)
print("messages",m0, m1)
c0 = enc(N, e, m0)
c1 = enc(N, e, m1)
print("encryptions", c0, c1)
b = random.randint(0,1)
if b == 0:
    c = enc(N, e, m0)
else: 
    c = enc(N, e, m1)
print("challenge encryption",c)    

pub 488489 65537
messages 378980 41295
encryptions 46995 28202
challenge encryption 46995


## Hybrid encryption

**Construction 10.12** 
* $\Pi = (Gen, Enc, Dec)$ a public-key encryption scheme
* $\Pi' = (Gen', Enc', Dec')$ a private-key encryption scheme
Construct a public-key encryption $\Pi^{hy} = (Gen^{hy}, Enc^{hy}, Dec^{hy})$:

* **Gen**${}^{hy}(1^n)$: $(pk, sk) \leftarrow Gen(1^n)$

* **Enc**${}^{hy}(pk, m):$
    1. $k \leftarrow \{0,1\}^n$
    2. 
        * $c_1 \leftarrow Enc_{pk}(k)$, 
        * $c_2 \leftarrow Enc'_k(m)$
    3. $(c_1, c_2)$

* **Dec**${}^{hy}(sk, c_1, c_2):$
    1. $k = Dec_{sk}(c_1)$
    2. $m = Dec'_{k}(c_2)$


## RSA OAEP


![oaep](img/oaep.png)

**Signature scheme (12.1)** a signature scheme is a tuple of three probabilistic polynomial-time algorithms $(Gen, Sign, Vrfy)$ satisfying the following:


1. $(pk, sk) \leftarrow Gen(1^n)$
2. $\sigma \leftarrow Sign_{sk}(m)$, $m \in \{0, 1\}^*$ ($Sign$ is probabilistic)
3. $$b  = Vrfy(pk, m, \sigma)$$
    * $b = 1$ valid 
    * $b = 0$ invalid

Signature experiment $SigForge_{\mathcal{A}, \Pi}(n)$:

1. $(pk, sk) \leftarrow Gen(1^n)$
2. 
    * $\mathcal{A}$ is given $pk$ and oracle access to $Sign_{sk}(\cdot)$
    * $\mathcal{A}$ outputs $(m, \sigma)$
    * $Q$ - the set of messages whose signature were requested by $\mathcal{A}$ during its execution
3. The output of the experiment is defined to be $1$ if and only if:
    1. $Vrfy_{pk}(m, \sigma) = 1$
    2. $m \not \in Q$

**Definition 12.2** A signature scheme $\Pi = (Gen, Sign, Vrfy)$ is **existentially unforgeable under an adaptive chosen-message attack** if for all probabilistic polynomial-time adversaries $\mathcal{A}$, there exists a negligible function **negl** such that
$$P\left[SigForge_{\mathcal{A}, \Pi}(n) = 1\right] \leq \textsf{negl}(n).$$

**Textbook RSA signature**

**Gen** $(N, e, d) \leftarrow GenRSA(1^n)$

* the public key $(N, e)$
* the private key $(N, d)$

**Sign**$([N, d], m)$, $m \in Z_N^*$:

* return $\sigma = [m^d \bmod N]$

**Vrfy**$([N, e], m, \sigma)$, output 1 if and only if

$$ m = [\sigma^e \bmod N]$$

In [56]:
def GenRSA(w):
    n = len(w)
    N, p, q = GenModulus(w)
    m = (p-1) * (q-1)
    e = 2 ** 16 + 1
    gcd, d, y = egcd(e, m)
    if d < 0:
        d = m + d
    return N, e, d

def sign(x, N, d):
    return x ** d % N

def vrfy(m, sigma, N, e):
    return m == sigma ** e % N

**Example 1** A no-message attack

Goal: forge a signature under a random message

$\mathcal{A}([N, e])$:
1. select $\sigma \leftarrow Z_N^*$
2. compute $m = \sigma^e \bmod N$
3. return $(m, \sigma)$

In [57]:
N, e, d = GenRSA("11111")

def A(N, e):
    sigma = randomZnElement(N)
    m = sigma ** e % N
    return m, sigma

In [58]:
m, sigma = A(N, e)
print(m, sigma)

1166 1996


In [59]:
vrfy(m, sigma, N, e)

True

Signature experiment $SigForge_{\mathcal{A}, \Pi}(n)$:

1. $(pk, sk) \leftarrow Gen(1^n)$
2. 
    * $\mathcal{A}$ is given $pk$ and oracle access to $Sign_{sk}(\cdot)$
    * $\mathcal{A}$ outputs $(m, \sigma)$
    * $Q$ - the set of messages whose signature were requested by $\mathcal{A}$ during its execution
3. The output of the experiment is defined to be $1$ if and only if:
    1. $Vrfy_{pk}(m, \sigma) = 1$
    2. $m \not \in Q$

## Textbook RSA signature

**Gen** $(N, e, d) \leftarrow GenRSA(1^n)$

* the public key $(N, e)$
* the private key $(N, d)$

**Sign**$([N, d], m)$, $m \in Z_N^*$:

* return $\sigma = [m^d \bmod N]$

**Vrfy**$([N, e], m, \sigma)$, output 1 if and only if

$$ m = [\sigma^e \bmod N]$$

**Example 2** Forging a signature on an arbitrary message

![forge](img/forge.png)

In [82]:
m = randomZnElement(N)
m1 = randomZnElement(N)
m2 = m * modinv(m1, N) % N

s1 = sign(m1, N, d)
s2 = sign(m2, N, d)

print(m, m1, m2)

print(s1, s2)


vrfy(m, (s1 * s2) % N, N, e)


1250 1946 1520
819 469


True

# Factoring - Fermat

In [77]:
p = randprime(2000, 10000)
q = randprime(2000, 10000)

n = p * q

print(p, q, n)


8353 6959 58128527


In [80]:
gcdPlus = 1
gcdMinus = 1

step = 1

while gcdPlus == 1 and gcdMinus == 1:
    x = random.randint(2, n)
    y = random.randint(2, n)
    gcdPlus = math.gcd((x+y) % n, n)
    gcdMinus = math.gcd((x-y) % n, n)
    print(step, gcdPlus, gcdMinus)
    step = step + 1
    
if gcdPlus > 1:
    print(gcdPlus,"*", n // gcdPlus, "=", n)
else:
    print(gcdMinus,"*", n // gcdMinus, "=", n)
    

1 1 1
2 1 1
3 1 1
4 1 1
5 1 1
6 1 1
7 1 1
8 1 1
9 1 1
10 1 1
11 1 1
12 1 1
13 1 1
14 1 1
15 1 1
16 1 1
17 1 1
18 1 1
19 1 1
20 1 1
21 1 1
22 1 1
23 1 1
24 1 1
25 1 1
26 1 1
27 1 1
28 1 1
29 1 1
30 1 1
31 1 1
32 1 1
33 1 1
34 1 1
35 1 1
36 1 1
37 1 1
38 1 1
39 1 1
40 1 1
41 1 1
42 1 1
43 1 1
44 1 1
45 1 1
46 1 1
47 1 1
48 1 1
49 1 1
50 1 1
51 1 1
52 1 1
53 1 1
54 1 1
55 1 1
56 1 1
57 1 1
58 1 1
59 1 1
60 1 1
61 1 1
62 1 1
63 1 1
64 1 1
65 1 1
66 1 1
67 1 1
68 1 1
69 1 1
70 1 1
71 1 1
72 1 1
73 1 1
74 1 1
75 1 1
76 1 1
77 1 1
78 1 1
79 1 1
80 1 1
81 1 1
82 1 1
83 1 1
84 1 1
85 1 1
86 1 1
87 1 1
88 1 1
89 1 1
90 1 1
91 1 1
92 1 1
93 1 1
94 1 1
95 1 1
96 1 1
97 1 1
98 1 1
99 1 1
100 1 1
101 1 1
102 1 1
103 1 1
104 1 1
105 1 1
106 1 1
107 1 1
108 1 1
109 1 1
110 1 1
111 1 1
112 1 1
113 1 1
114 1 1
115 1 1
116 1 1
117 1 1
118 1 1
119 1 1
120 1 1
121 1 1
122 1 1
123 1 1
124 1 1
125 1 1
126 1 1
127 1 1
128 1 1
129 1 1
130 1 1
131 1 1
132 1 1
133 1 1
134 1 1
135 1 1
136 1 1
137 1 1
138 1 1
139 

1225 1 1
1226 1 1
1227 1 1
1228 1 1
1229 1 1
1230 1 1
1231 1 1
1232 1 1
1233 1 1
1234 1 1
1235 1 1
1236 1 1
1237 1 1
1238 1 1
1239 1 1
1240 1 1
1241 1 1
1242 1 1
1243 1 1
1244 1 1
1245 1 1
1246 1 1
1247 1 1
1248 1 1
1249 1 1
1250 1 1
1251 1 1
1252 1 1
1253 1 1
1254 1 1
1255 1 1
1256 1 1
1257 1 1
1258 1 1
1259 1 1
1260 1 1
1261 1 1
1262 1 1
1263 1 1
1264 1 1
1265 1 1
1266 1 1
1267 1 1
1268 1 1
1269 1 1
1270 1 1
1271 1 1
1272 1 1
1273 1 1
1274 1 1
1275 1 1
1276 1 1
1277 1 1
1278 1 1
1279 1 1
1280 1 1
1281 1 1
1282 1 1
1283 1 1
1284 1 1
1285 1 1
1286 1 1
1287 1 1
1288 1 1
1289 1 1
1290 1 1
1291 1 1
1292 1 1
1293 1 1
1294 1 1
1295 1 1
1296 1 1
1297 1 1
1298 1 1
1299 1 1
1300 1 1
1301 1 1
1302 1 1
1303 1 1
1304 1 1
1305 1 1
1306 1 1
1307 1 1
1308 1 1
1309 1 1
1310 1 1
1311 1 1
1312 1 1
1313 1 1
1314 1 1
1315 1 1
1316 1 1
1317 1 1
1318 1 1
1319 1 1
1320 1 1
1321 1 1
1322 1 1
1323 1 1
1324 1 1
1325 1 1
1326 1 1
1327 1 1
1328 1 1
1329 1 1
1330 1 1
1331 1 1
1332 1 1
1333 1 1
1334 1 1
1335 1 1
1

2519 1 1
2520 1 1
2521 1 1
2522 1 1
2523 1 1
2524 1 1
2525 1 1
2526 1 1
2527 1 1
2528 1 1
2529 1 1
2530 1 1
2531 1 1
2532 1 1
2533 1 1
2534 1 1
2535 1 1
2536 1 1
2537 1 1
2538 1 1
2539 1 1
2540 1 1
2541 1 1
2542 1 1
2543 1 1
2544 1 1
2545 1 1
2546 1 1
2547 1 1
2548 1 1
2549 1 1
2550 1 1
2551 1 1
2552 1 1
2553 1 1
2554 1 1
2555 1 1
2556 1 1
2557 1 1
2558 1 1
2559 1 1
2560 1 1
2561 1 1
2562 1 1
2563 1 1
2564 1 1
2565 1 1
2566 1 1
2567 1 1
2568 1 1
2569 1 1
2570 1 1
2571 1 1
2572 1 1
2573 1 1
2574 1 1
2575 1 1
2576 1 1
2577 1 1
2578 1 1
2579 1 1
2580 1 1
2581 1 1
2582 1 1
2583 1 1
2584 1 1
2585 1 1
2586 1 1
2587 1 1
2588 1 1
2589 1 1
2590 1 1
2591 1 1
2592 1 1
2593 1 1
2594 1 1
2595 1 1
2596 1 1
2597 1 1
2598 1 1
2599 1 1
2600 1 1
2601 1 1
2602 1 1
2603 1 1
2604 1 1
2605 1 1
2606 1 1
2607 1 1
2608 1 1
2609 1 1
2610 1 1
2611 1 1
2612 1 1
2613 1 1
2614 1 1
2615 1 1
2616 1 1
2617 1 1
2618 1 1
2619 1 1
2620 1 1
2621 1 1
2622 1 1
2623 1 1
2624 1 1
2625 1 1
2626 1 1
2627 1 1
2628 1 1
2629 1 1
2